# 03.Entrenamiento del modelo

In [6]:
pip install spacy nltk

   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.2 MB 3.4 MB/s eta 0:00:04
   ------------------------- -------------- 7.9/12.2 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.2 MB 21.6 MB/s eta 0:00:01
   ---------------------------------------- 12.2/12.2 MB 18.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 20.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/632.6 kB ? eta -:--:--
   ---------------------------------------- 632.6/632.6 kB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 48.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   -------------------------------------- - 6.0/6.2 MB 33.7 MB/s eta 0:00:01
   -------------------------

In [17]:
import sqlite3
import datetime
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\equipo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\equipo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importacion de la Data

In [ ]:


# Guardar en un dataframe
def save_to_dataframe(db_path, table_name):
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect(db_path)
    
    # Leer la tabla en un DataFrame de pandas
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    
    # Cerrar la conexión
    conn.close()
    
    return df




### Noticias

In [2]:
df_news=save_to_dataframe('../Datos/data.db', 'news')
df_news.head(10)

,fecha,fuente,titulo,descripcion,url
0,20/04/2025,Eldiario.es,Del huerto urbano a los mejores restaurantes: ...,Un proyecto para sacar de la exclusión a gente...,https://www.eldiario.es/ballenablanca/economia...
1,20/04/2025,Criptonoticias.com,Solana lleva la batuta del alza esta semana y ...,"Solana lidera con alza del 20% a USD 136,50. S...",https://www.criptonoticias.com/mercados/solana...
2,20/04/2025,Eldiario.es,El precio del plátano sigue sin techo: el cons...,"Los cosecheros locales, según registros del MA...",https://www.eldiario.es/canariasahora/eldiario...
3,20/04/2025,Forbes.com.mx,Empobrecer al prójimo es perjudicarse: arancel...,Forbes México.\n Empobrecer al prójimo es perj...,https://forbes.com.mx/empobrecer-al-projimo-es...
4,20/04/2025,Elespanol.com,Replantearse las represalias por los aranceles...,¿Y si Estados Unidos no estuviera rompiendo la...,https://www.elespanol.com/invertia/opinion/202...
5,20/04/2025,La Nacion,El traslado a precios del nuevo dólar aún no s...,Distintas consultoras detectaron movimientos s...,https://www.lanacion.com.ar/economia/el-trasla...
6,19/04/2025,Cointelegraph,El equilibrio del inversor: Rentabilidad y con...,"El inversor inteligente controla gastos, anali...",https://es.cointelegraph.com/news/the-investor...
7,19/04/2025,La Nacion,El imperio del dólar barato,Milei apela a la táctica del miedo para limita...,https://www.lanacion.com.ar/politica/el-imperi...
8,19/04/2025,La Nacion,La oposición quiere apurar un proyecto que ext...,El objetivo es evitar que vuelvan a subir en j...,https://www.lanacion.com.ar/politica/la-oposic...
9,18/04/2025,La Nacion,Maximiliano Pullaro no tuvo suerte,Las elecciones en Santa Fe dicen mucho sobre l...,https://www.lanacion.com.ar/ideas/maximiliano-...


### Historico

In [3]:
df_hist=save_to_dataframe('../Datos/data.db', 'historic')
df_hist.head(100)

,Close_COP=X,High_COP=X,Low_COP=X,Open_COP=X,Volume_COP=X,SMA_30
0,4045.770020,4050.040039,4041.000000,4045.770020,0,NaN
1,4026.000000,4032.250000,4006.030029,4029.250000,0,NaN
2,4020.239990,4043.110107,4018.939941,4020.239990,0,NaN
3,4039.750000,4040.219971,4039.000000,4039.750000,0,NaN
4,4033.689941,4058.820068,4015.165283,4033.689941,0,NaN
...,...,...,...,...,...,...
95,3674.100098,3685.260010,3642.520020,3674.100098,0,3756.004997
96,3656.250000,3666.629883,3617.528564,3656.250000,0,3756.757666
97,3668.100098,3713.729980,3647.331787,3668.100098,0,3756.796004
98,3713.260010,3713.260010,3672.768311,3713.260010,0,3757.589673


# Analisis de sentimientos

In [4]:
df_sentiment=df_news[['fecha','titulo']]
df_sentiment.head()

,fecha,titulo
0,20/04/2025,Del huerto urbano a los mejores restaurantes: ...
1,20/04/2025,Solana lleva la batuta del alza esta semana y ...
2,20/04/2025,El precio del plátano sigue sin techo: el cons...
3,20/04/2025,Empobrecer al prójimo es perjudicarse: arancel...
4,20/04/2025,Replantearse las represalias por los aranceles...


In [11]:
stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')
def preprocess_text(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    doc = nlp(texto)
    tokens = [token.text for token in doc if token.is_alpha]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)

In [18]:
df_sentiment['texto_procesado'] = df_sentiment['titulo'].apply(preprocess_text)

C:\Users\equipo\AppData\Local\Temp\ipykernel_4272\3133770228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['texto_procesado'] = df_sentiment['titulo'].apply(preprocess_text)


In [19]:
df_sentiment.head()

,fecha,titulo,texto_procesado
0,20/04/2025,Del huerto urbano a los mejores restaurantes: ...,huert urban mejor restaur vancouv revolucion a...
1,20/04/2025,Solana lleva la batuta del alza esta semana y ...,solan llev batut alza seman proxim preci clav usd
2,20/04/2025,El precio del plátano sigue sin techo: el cons...,preci platan sig tech consumidor pag cuatr eur...
3,20/04/2025,Empobrecer al prójimo es perjudicarse: arancel...,empobrec projim perjudic arancel trump conllev...
4,20/04/2025,Replantearse las represalias por los aranceles...,replant represali arancel trump trat realment ...


In [21]:
# Implementacion transformers para sentimiento
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


# Aplicar análisis de sentimiento y guardar los resultados
resultados = df_sentiment["texto_procesado"].apply(lambda x: sentiment_pipeline(x)[0])

Device set to use cpu


In [22]:
df_sentiment['puntaje']=resultados.apply(lambda x: x["score"])

C:\Users\equipo\AppData\Local\Temp\ipykernel_4272\1764502434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['puntaje']=resultados.apply(lambda x: x["score"])


In [24]:
df_sentiment['puntaje']=df_sentiment['puntaje'].apply(lambda x: 2*x-1)

C:\Users\equipo\AppData\Local\Temp\ipykernel_4272\914804743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['puntaje']=df_sentiment['puntaje'].apply(lambda x: 2*x-1)


In [25]:
df_sentiment.head(10)

,fecha,titulo,texto_procesado,puntaje
0,20/04/2025,Del huerto urbano a los mejores restaurantes: ...,huert urban mejor restaur vancouv revolucion a...,0.038727
1,20/04/2025,Solana lleva la batuta del alza esta semana y ...,solan llev batut alza seman proxim preci clav usd,0.069671
2,20/04/2025,El precio del plátano sigue sin techo: el cons...,preci platan sig tech consumidor pag cuatr eur...,-0.168782
3,20/04/2025,Empobrecer al prójimo es perjudicarse: arancel...,empobrec projim perjudic arancel trump conllev...,-0.110078
4,20/04/2025,Replantearse las represalias por los aranceles...,replant represali arancel trump trat realment ...,0.637302
5,20/04/2025,El traslado a precios del nuevo dólar aún no s...,trasl preci nuev dol aun percib relev aliment beb,-0.380857
6,19/04/2025,El equilibrio del inversor: Rentabilidad y con...,equilibri inversor rentabil control gast,-0.173900
7,19/04/2025,El imperio del dólar barato,imperi dol barat,-0.385334
8,19/04/2025,La oposición quiere apurar un proyecto que ext...,oposicion quier apur proyect extiend baj reten...,-0.150301
9,18/04/2025,Maximiliano Pullaro no tuvo suerte,maximilian pullar suert,-0.105957


In [26]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fecha            42 non-null     object 
 1   titulo           42 non-null     object 
 2   texto_procesado  42 non-null     object 
 3   puntaje          42 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.4+ KB


In [27]:
df_sentiment['fecha'].value_counts()

fecha
20/04/2025    6
19/04/2025    3
18/04/2025    3
17/04/2025    3
16/04/2025    3
15/04/2025    3
14/04/2025    3
13/04/2025    3
12/04/2025    3
11/04/2025    3
10/04/2025    3
09/04/2025    3
08/04/2025    3
Name: count, dtype: int64

# Dataframe para el modelo

In [ ]:
df_model